<h1>Plausibility check of crowdsourced data

In [ ]:
import os
from scipy.stats import ks_2samp
import pandas as pd
import json
import numpy as np

json_path = "E:\MA_data\WPPs+production\WPPs+production_new_complete.json"

with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

all_normalised_productions = []

for entry in data:
    capacity = entry.get("Capacity")
    production_data = entry.get("Production")

    for timestamp, value in production_data:
        if isinstance(value, (int, float)) and capacity > 0:
            all_normalised_productions.append(value / capacity)

# In numpy umwandeln
arr = np.array(all_normalised_productions)

# Erwartungswert und Standardabweichung berechnen
mean = arr.mean()
std = arr.std()

print("Original dataset")
print(f"    number of normalised values: {len(arr)}")
print(f"    expectation value: {mean:.4f}")
print(f"    standard deviation: {std:.4f}")


input_dir = "data/crowdsourced_data/"

for file in os.listdir(input_dir):
    if file.endswith(".xlsx"):
        print(f"Dataset {file}")
        file_path = os.path.join(input_dir, file)
        sheets = pd.read_excel(file_path, sheet_name=["Time Series", "Metadata"])
        time_series_data = sheets["Time Series"]
        capacity = sheets["Metadata"].iloc[0]["Capacity (MW)"]
        productions = time_series_data["Production (MW)"]
        norm_productions_crs = productions / capacity

        total = len(norm_productions_crs.dropna())
        below_zero_pct = (norm_productions_crs < 0).sum() / total * 100
        above_max_pct = (norm_productions_crs > 1.05).sum() / total * 100
        std_crs = norm_productions_crs.std()
        var_sim = std_crs**2 / std**2 # Variance similarity
        mean_crs = norm_productions_crs.mean()
        mean_sim = mean_crs / mean # Mean similarity

        if below_zero_pct > 0.01:
            print(f"    Discard dataset: {below_zero_pct} % of normalised power values below 0")

        if above_max_pct > 0.05:
            print(f"    Discard dataset: {above_max_pct} % of normalised power values above 1.05")

        if var_sim > 1.5:
            print(f"    Discard dataset: Standard deviation = {std_crs} too high")

        if var_sim < 0.5:
            print(f"    Discard dataset: Standard deviation = {std_crs} too low")

        if mean_sim > 1.5:
            print(f"    Discard dataset: Mean = {mean_crs} too high")

        if mean_sim < 0.5:
            print(f"    Discard dataset: Mean = {mean_crs} too low")

        # Kolmogorov–Smirnov-Test (KS-Test)
        statistic, p_value = ks_2samp(all_normalised_productions, norm_productions_crs)
        print(f"statistic: {statistic}, p_value: {p_value}")

example crowdsourced data not very representative

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.hist(all_normalised_productions, bins=100, alpha=0.5, label="Reference", density=True)
plt.hist(norm_productions_crs, bins=100, alpha=0.5, label="Crowdsourced", density=True)
plt.xlabel("Normalised Production")
plt.ylabel("Density")
plt.title("Distribution Comparison")
plt.legend()
plt.tight_layout()
plt.show()


<h1>(Manually delete crowdsourced data that doesn't seem plausible)

<h1>Load crowdsourced data and fetch according wind data

In [ ]:
import os
import pandas as pd
import cdsapi  # for reanalysis data
from ecmwfapi import ECMWFService  # for reforecast data

input_dir = "data/crowdsourced_data/"
output_dir = "data/crowdsourcing_wind"

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

for file in os.listdir(input_dir):
    if file.endswith(".xlsx"):
        file_path = os.path.join(input_dir, file)
        sheets = pd.read_excel(file_path, sheet_name=["Time Series", "Metadata"])
        time_series_data = sheets["Time Series"]
        metadata = sheets["Metadata"]

        timestamp = file.removeprefix("time_series_").removesuffix(".xlsx")

        time_series_data["Date"] = pd.to_datetime(time_series_data["Date"], errors='coerce')

        # Extract unique years, months, days, and hours
        years = sorted(set(time_series_data["Date"].dt.year.dropna().astype(str)))
        months = sorted(set(time_series_data["Date"].dt.month.dropna().astype(str).str.zfill(2)))
        days = sorted(set(time_series_data["Date"].dt.day.dropna().astype(str).str.zfill(2)))
        hours = sorted(set(time_series_data["Date"].dt.hour.dropna().astype(str).str.zfill(2)))

        # Extract location
        lat, lon = metadata.iloc[0]["Latitude"], metadata.iloc[0]["Longitude"]

        # =================== REANALYSIS DATA (ERA5) ===================

        # # API key saved in .cdsapirc
        # client = cdsapi.Client()
        # reanalysis_file = os.path.join(output_dir, f"reanalysis_{timestamp}.grib")

        # if not os.path.exists(reanalysis_file):  # Skip if already downloaded
        #     print(f"Downloading ERA5 reanalysis data for {timestamp}...")

        #     request = {
        #         "product_type": "reanalysis",
        #         "variable": ["100m_u_component_of_wind", "100m_v_component_of_wind"],
        #         "year": years,
        #         "month": months,
        #         "day": days,
        #         "time": hours,
        #         "format": "grib",
        #         "area": [lat+1, lon-1, lat-1, lon+1],  # N/W/S/E
        #     }

        #     client.retrieve("reanalysis-era5-single-levels", request, reanalysis_file)
        #     print(f"Saved reanalysis data: {reanalysis_file}")
        # else:
        #     print(f"Skipping ERA5 reanalysis data for {timestamp}, already exists.")

        # =================== REFORECAST DATA (ECMWF) ===================

        # API key under https://api.ecmwf.int/v1/key/, saved in .ecmwfapirc
        server = ECMWFService("mars")
        reforecast_file = os.path.join(output_dir, f"reforecast_{timestamp}.grib")

        if not os.path.exists(reforecast_file):  # Skip if already downloaded
            print(f"Downloading ECMWF reforecast data for {timestamp}...")

            server.execute( # ignore efficient MARS request guidelines here, since process should be automated and available timestamps cannot be analysed individually
                {
                    "class": "od",
                    "dataset": "oper",
                    "expver": "1",
                    "stream": "oper",
                    "type": "fc",
                    "levtype": "sfc",  # surface-level type
                    "param": "246.228/247.228",  # u100 and v100
                    "date": f"{min(years)}-{min(months)}-01/to/{max(years)}-{max(months)}-31",  # Ensure correct format
                    "time": ["00:00", "12:00"],  # Forecast times
                    "step": [str(i) for i in range(0, 145, 3)],  # Convert to list of strings
                    "grid": "0.25/0.25",
                    "area": [lat, lon, lat, lon],  # N/W/S/E
                    "format": "grib2",
                },
                reforecast_file
            )
            print(f"Saved reforecast data: {reforecast_file}")
        else:
            print(f"Skipping ECMWF reforecast data for {timestamp}, already exists.")

print("All downloads completed.")

<h1>Build one dictionary for all WPPs (part I), and for each WPP, assign wind data (part II)

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import xarray as xr
from scipy.interpolate import RegularGridInterpolator
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Define lead times
lead_times = list(range(0, 145, 3))

# Directories
input_dir = "data/crowdsourced_data"
forecast_dir = "data/crowdsourcing_wind"
output_dir = "data/crowdsourcing_wind_production/json_lead_times"

os.makedirs(output_dir, exist_ok=True)

# Load crowdsourced WPP production data
wpp_files = [f for f in os.listdir(input_dir) if f.endswith(".xlsx")]
wpps = []

for file in wpp_files:
    file_path = os.path.join(input_dir, file)
    sheets = pd.read_excel(file_path, sheet_name=["Time Series", "Metadata"])
    time_series_data = sheets["Time Series"]
    metadata = sheets["Metadata"]

    time_series_data["Date"] = pd.to_datetime(time_series_data["Date"], errors='coerce')
    
    upload_time = file.replace('time_series_', '').replace('.xlsx', '')
    lat = metadata.iloc[0]["Latitude"]
    lon = metadata.iloc[0]["Longitude"]
    turbine_type = metadata.iloc[0]["Turbine Type"]
    hub_height = metadata.iloc[0]["Hub Height"]
    comm_year = metadata.iloc[0]["Commissioning Year"]
    comm_month = metadata.iloc[0]["Commissioning Month"]
    capacity = metadata.iloc[0]["Capacity (MW)"]

    # designation of keys just like in existing dataset II!
    wpps.append({
        "Upload Time": upload_time,
        "Latitude": metadata.iloc[0]["Latitude"],
        "Longitude": metadata.iloc[0]["Longitude"],
        "Turbine": metadata.iloc[0]["Turbine Type"],
        "Hub_height": metadata.iloc[0]["Hub Height"],
        "Commissioning_date": f"{int(metadata.iloc[0]["Commissioning Year"])}/{int(metadata.iloc[0]["Commissioning Month"])}",
        "Capacity": metadata.iloc[0]["Capacity (MW)"],
        "Production": time_series_data.values.tolist()  # Store entire time series
    })

grib_files = sorted([f for f in os.listdir(forecast_dir) if f.endswith(".grib")])

# Process each reforecast file
for grib_file in grib_files:
    
    grib_path = os.path.join(forecast_dir, grib_file)

    # Load wind speed forecast data
    try:
        ds = xr.open_dataset(grib_path, engine="cfgrib", chunks={"time": 100})
    except Exception as e:
        print(f"Error reading {grib_file}: {e} --> skipping")
        continue

    upload_time = grib_file.replace('reforecast_', '').replace('.grib', '')

    file_path = os.path.join(input_dir, f"{grib_file.replace('reforecast', 'time_series').replace('grib', 'xlsx')}")
    matching_wpps = [wpp for wpp in wpps if wpp["Upload Time"] == upload_time]
    wpp = matching_wpps[0]

    lead_time_dicts = {str(lt): {} for lt in lead_times}

    # Output JSON file path for this month-year
    output_json_file = os.path.join(output_dir, f"{grib_file.replace('.grib', '_wind.json')}")

    # Check if this month's data has already been processed (only when this cell is restarted before execution has finished)
    if os.path.exists(output_json_file):
        print(f"Skipping {output_json_file}, already exists.")
        continue

    print(f"Processing {grib_file}...")

    # Extract forecast timestamps and wind components
    times = pd.to_datetime(ds["valid_time"].values)
    latitudes = ds["latitude"].values
    longitudes = ds["longitude"].values
    u = ds["u100"].values
    v = ds["v100"].values

    unique_key = wpp["Upload Time"]  # Unique identifier of a wpp
    lon = wpp["Longitude"]
    lat = wpp["Latitude"]
    time_series = wpp["Production"]

    forecast_data = []

    for time_series_value in time_series:
        timestep = time_series_value[0]
        production = time_series_value[1]
        for j in range(len(times)):
            forecast_times = times[j]
            if timestep in forecast_times: # crowdsourced timesteps that are not dividable by 3 (resolution of reforecsts) have to be discarded
                forecast_times = pd.DatetimeIndex(forecast_times) # convert from DatetimeArray to DatetimeIndex, because only this one has .get_loc()
                time_index = forecast_times.get_loc(timestep)
                lead_time = lead_times[time_index]

                forecast_u = u[j]
                forecast_v = v[j]

                wind_speeds = np.sqrt(forecast_u[time_index]**2 + forecast_v[time_index]**2)

                if latitudes[0] < latitudes[-1]:
                    latitudes = latitudes[::-1]
                    wind_speeds = wind_speeds[::-1, :]

                interpolator = RegularGridInterpolator(
                    (latitudes, longitudes), wind_speeds, method="linear", bounds_error=False, fill_value=None
                )

                wind_speed_value = interpolator([[lat, lon]])[0]

                wind_speed_value = round(wind_speed_value, 2)

                forecast_data.append([lead_time, timestep, production, wind_speed_value])

    forecast_df = pd.DataFrame(forecast_data, columns=["lead_time", "forecast_time", "production", "wind_speed"])

    # Store data in structured dictionary grouped by lead time
    for lead_time in lead_times:
        lead_time_str = str(lead_time)  # Ensure keys are strings for JSON compatibility

        lead_time_mask = forecast_df["lead_time"] == lead_time
        lead_time_data = forecast_df[lead_time_mask]

        if lead_time_data.empty:
            continue

        lead_time_dicts[lead_time_str][unique_key] = {
            "Upload_time": str(wpp["Upload Time"]),
            "Capacity": float(wpp["Capacity"]),
            "Latitude": float(wpp["Latitude"]),
            "Longitude": float(wpp["Longitude"]),
            "Turbine": str(wpp["Turbine"]),
            "Hub_height": float(wpp["Hub_height"]),
            "Commissioning_date": str(wpp["Commissioning_date"]),
            "Time Series": [
                [str(date), float(production), float(wind_speed)]
                for _, date, production, wind_speed in lead_time_data[["lead_time", "forecast_time", "production", "wind_speed"]].values.tolist()
            ]
        }


    # Save updated JSON
    with open(output_json_file, "w", encoding="utf-8") as f:
        json.dump(lead_time_dicts, f, indent=4)

    print(f"Updated JSON file saved: {output_json_file}")

<h1>Merge all files into one per lead time

In [ ]:
import os
import json
from collections import defaultdict

# Path to input JSON files
input_json_dir = "data/crowdsourcing_wind_production/json_lead_times"

# Dictionary to hold all data sorted by lead time
lead_time_dicts = defaultdict(lambda: defaultdict(dict))

# Load all JSON files
json_files = [f for f in os.listdir(input_json_dir) if f.endswith("_wind.json")]

for json_file in json_files:
    json_path = os.path.join(input_json_dir, json_file)
    print(f"Loading {json_file}")

    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    # Iterate over lead times in the loaded JSON
    for lead_time, wpp_data in data.items():
        for unique_key, wpp_entry in wpp_data.items():
            if unique_key in lead_time_dicts[lead_time]:
                # **Merge production data if the WPP already exists**
                lead_time_dicts[lead_time][unique_key]["Time Series"].extend(wpp_entry["Time Series"])
            else:
                # **Otherwise, add the entire WPP entry**
                lead_time_dicts[lead_time][unique_key] = wpp_entry

# Save merged data into separate files per lead time
output_dir = "data/crowdsourcing_wind_production"
os.makedirs(output_dir, exist_ok=True)

for lead_time, wpp_entries in lead_time_dicts.items():
    output_file = os.path.join(output_dir, f"WPPs+production+wind_lead_time_{lead_time}.json")
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(wpp_entries, f, indent=4)
    print(f"Saved merged file: {output_file}")

<h1>Second plausibility test to verify crowdsourced data: old model's predictions

In [ ]:
import numpy as np
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import joblib
import os
import pandas as pd
from torch.nn import HuberLoss

# Lists to store models and scalers
encoders = joblib.load("modelC/parameters/encoders.pkl")
input_sizes = joblib.load("modelC/parameters/input_sizes.pkl")
scalers = joblib.load("modelC/parameters/scalers.pkl")
test_indices = joblib.load("modelC/parameters/test_indices.pkl") # same random_split as during training of the model, so that testing now only is done on unseen data 
model_state_dicts = torch.load("modelC/parameters/models.pth", weights_only=True)

# Define MLP class
class MLP(nn.Module):
    def __init__(self, input_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 1)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        self.relu3 = nn.ReLU()
        self.dropout = nn.Dropout(0.3366)

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.relu3(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        return x
    
metrics = {}

# PyTorch Dataset Class
class WindPowerDataset(Dataset):
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, index):
        x = self.features[index]
        y = self.targets[index]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

input_dir = r"E:\MA_data\WPPs+production+reforecast"

for file in os.listdir(input_dir):
    file_path = os.path.join(input_dir, file)
    if os.path.isfile(file_path):  # Ensure it's a file (not a folder)
        lead_time = int(file.split("_")[-1].replace(".json", ""))
        with open(file_path, "r", encoding="utf-8") as file:
            forecast_data = json.load(file)

    print(f"Processing lead time: {lead_time}")

    print(f"    Data preparation")

    all_turbine_types = []
    all_hub_heights = []
    all_capacities = []
    all_commissioning_dates = []
    all_production_data = []

    for unique_key, wpp in forecast_data.items():
        all_turbine_types.append(str(wpp["Turbine"]))
        all_hub_heights.append(wpp["Hub_height"])
        all_capacities.append(wpp["Capacity"])
        all_commissioning_dates.append(f"{wpp['Commissioning_date']}/06" if isinstance(wpp["Commissioning_date"], str) and "/" not in wpp["Commissioning_date"] else wpp["Commissioning_date"])
        all_production_data.append(wpp["Time Series"])

    # One-Hot-Encoding for turbine types
    encoder = OneHotEncoder(sparse_output=False)
    turbine_types_onehot = encoder.fit_transform(np.array(all_turbine_types).reshape(-1, 1))

    # convert to datetime
    standardised_dates = pd.to_datetime(all_commissioning_dates, format='%Y/%m')

    # calculate age
    ref_date = pd.Timestamp("2024-12-01")
    ages = ref_date.year * 12 + ref_date.month - (standardised_dates.year * 12 + standardised_dates.month)

    # create combined features and output lists
    combined_features_raw = []
    output_raw = []
    
    # convert data in feature arrays
    for idx, production_data in enumerate(all_production_data):
        num_rows = len(production_data)

        # Repetitions for common features
        turbine_type_repeated = np.tile(turbine_types_onehot[idx], (num_rows, 1))
        hub_height_repeated = np.full((num_rows, 1), float(all_hub_heights[idx]))
        age_repeated = np.full((num_rows, 1), ages[idx])

        # Extract production values and wind speeds
        production_values = np.array([entry[1] for entry in production_data]).reshape(-1, 1) / all_capacities[idx]
        wind_speeds = np.array([entry[2] for entry in production_data]).reshape(-1, 1)

        # combine all features
        combined_chunk = np.hstack((
            turbine_type_repeated,
            hub_height_repeated,
            age_repeated,
            wind_speeds
        ))

        # add the data
        combined_features_raw.append(combined_chunk)
        output_raw.append(production_values)

    # combine all data chunks to one array
    combined_features = np.vstack(combined_features_raw)
    output = np.vstack(output_raw)

    # Interpolate missing values (linear interpolation) in pandas
    wind_speed_series = pd.Series(combined_features[:, -1])
    wind_speed_series.interpolate(method='linear', inplace=True)
    combined_features[:, -1] = wind_speed_series.to_numpy()

    # round all values to four decimal places
    combined_features = np.round(combined_features, decimals=4)
    output = np.round(output, decimals=4)
        
    # Normalise numerical features
    scaler_wind = StandardScaler()
    scaler_ages = StandardScaler()
    scaler_hub_heights = StandardScaler()

    # Skalieren der einzelnen Features
    combined_features[:, -1] = scaler_wind.fit_transform(combined_features[:, -1].reshape(-1, 1)).flatten() # scale wind speeds
    combined_features[:, -2] = scaler_ages.fit_transform(combined_features[:, -2].reshape(-1, 1)).flatten()  # scale ages
    combined_features[:, -3] = scaler_hub_heights.fit_transform(combined_features[:, -3].reshape(-1, 1)).flatten()  # scale hub heights
    
    # Convert to PyTorch Dataset
    dataset = WindPowerDataset(combined_features, output)
    
    params = {"batch_size": 128,
              "lr": 0.00010155,
              "number_epochs": 10}
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # shuffling matters here
    data_loader = DataLoader(dataset, batch_size=params["batch_size"], shuffle=True)
    
    # Model setup
    input_size = combined_features.shape[1]

    # use static instead of dynamic computational graphs
    model = torch.jit.script(MLP(input_size=input_size)).to(device)
    model.to(device)
    
    # Trainings-Konfiguration
    huber_criterion = HuberLoss()
    optimizer = optim.Adam(model.parameters(), lr=params["lr"])

    # Testen
    print(f"    Testing")
    model.eval()

    test_loss_mae, test_loss_mse, test_loss_huber = 0, 0, 0

    with torch.no_grad():
        for batch_x, batch_y in data_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            preds = model(batch_x)
            
            test_loss_huber += huber_criterion(preds, batch_y).item()

    test_loss_huber /= len(data_loader)
    
    metrics[lead_time] = {
        "Huber": test_loss_huber
    }

print(metrics)

<h1>(In case of too high errors with the old model, the data could be discarded as well. However, this needs some closer considerations.)

<h1>Extend dataset by verified crowdsourced data

In [ ]:
import os
import json
import datetime

# Aktueller Zeitstempel
current_utc = datetime.datetime.now(datetime.timezone.utc).strftime("%Y%m%d_%H%M%S")

# Verzeichnisse
old_data_path_1 = r"E:\MA_data\WPPs+production+reforecast"
old_data_path_2 = "data/crowdsourcing_wind_production"
new_data_path = rf"E:\MA_data\WPPs+production+reforecast_crs_{current_utc}"

# Zielverzeichnis erstellen, falls nicht vorhanden
os.makedirs(new_data_path, exist_ok=True)

# Korrektur: os.listdir (nicht os.listfir)
for file in os.listdir(old_data_path_1):
    file_path_1 = os.path.join(old_data_path_1, file)
    if not os.path.isfile(file_path_1):
        continue

    # Führt nur `.json`-Dateien weiter aus
    if not file.endswith(".json"):
        continue

    # Extrahiere lead_time
    try:
        lead_time = int(file.split("_")[-1].replace(".json", ""))
    except ValueError:
        print(f"Dateiname nicht gültig für lead_time: {file}")
        continue

    with open(file_path_1, "r", encoding="utf-8") as f1:
        data1 = json.load(f1)

    print(f"Processing lead time: {lead_time}")

    # Passende Datei im zweiten Verzeichnis suchen
    file2_name = f"WPPs+production+wind_lead_time_{lead_time}.json"
    file_path_2 = os.path.join(old_data_path_2, file2_name)

    # Nur zusammenführen, wenn die zweite Datei auch existiert
    if os.path.exists(file_path_2):
        with open(file_path_2, "r", encoding="utf-8") as f2:
            data2 = json.load(f2)

        # Beide Dictionaries kombinieren (überschreibt Schlüssel bei Konflikt – ggf. anpassen)
        combined_data = {**data1, **data2}
    else:
        combined_data = data1  # Nur Daten aus Pfad 1

    # Datei im Zielverzeichnis speichern
    output_path = os.path.join(new_data_path, f"WPPs+production+wind_lead_time_{lead_time}.json")
    with open(output_path, "w", encoding="utf-8") as out_file:
        json.dump(combined_data, out_file, indent=4)

    print(f"Saved combined data to {output_path}")

<h1>Retrain from scratch with extended dataset

In [ ]:
import numpy as np
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import joblib
import os
import pandas as pd
from torch.nn import HuberLoss, MSELoss, L1Loss

# Lists to store models and scalers
models = {}
scalers = {}
encoders = {}
input_sizes = {}
metrics = {}

# Define MLP class
class MLP(nn.Module):
    def __init__(self, input_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 1)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        self.relu3 = nn.ReLU()
        self.dropout = nn.Dropout(0.3366)

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.relu3(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        return x

# PyTorch Dataset Class
class WindPowerDataset(Dataset):
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, index):
        x = self.features[index]
        y = self.targets[index]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

input_dir = r"E:\MA_data\WPPs+production+reforecast_crs_20250328_154208"

for file in os.listdir(input_dir):
    file_path = os.path.join(input_dir, file)
    if not os.path.isfile(file_path):  # Ensure it's a file (not a folder)
        continue

    lead_time = int(file.split("_")[-1].replace(".json", ""))
    with open(file_path, "r", encoding="utf-8") as file:
        forecast_data = json.load(file)

    print(f"Processing lead time: {lead_time}")

    print(f"    Data preparation")

    all_turbine_types = []
    all_hub_heights = []
    all_capacities = []
    all_commissioning_dates = []
    all_production_data = []

    for unique_key, wpp in forecast_data.items():
        all_turbine_types.append(str(wpp["Turbine"]))
        all_hub_heights.append(wpp["Hub_height"])
        all_capacities.append(wpp["Capacity"])
        all_commissioning_dates.append(f"{wpp['Commissioning_date']}/06" if isinstance(wpp["Commissioning_date"], str) and "/" not in wpp["Commissioning_date"] else wpp["Commissioning_date"])
        all_production_data.append(wpp["Time Series"])

    # One-Hot-Encoding for turbine types
    encoder = OneHotEncoder(sparse_output=False)
    turbine_types_onehot = encoder.fit_transform(np.array(all_turbine_types).reshape(-1, 1))

    # convert to datetime
    standardised_dates = pd.to_datetime(all_commissioning_dates, format='%Y/%m')

    # calculate age
    ref_date = pd.Timestamp("2024-12-01")
    ages = ref_date.year * 12 + ref_date.month - (standardised_dates.year * 12 + standardised_dates.month)

    # create combined features and output lists
    combined_features_raw = []
    output_raw = []
    
    # convert data in feature arrays
    for idx, production_data in enumerate(all_production_data):
        num_rows = len(production_data)

        # Repetitions for common features
        turbine_type_repeated = np.tile(turbine_types_onehot[idx], (num_rows, 1))
        hub_height_repeated = np.full((num_rows, 1), float(all_hub_heights[idx]))
        age_repeated = np.full((num_rows, 1), ages[idx])

        # Extract production values and wind speeds
        production_values = np.array([entry[1] for entry in production_data]).reshape(-1, 1) / all_capacities[idx]
        wind_speeds = np.array([entry[2] for entry in production_data]).reshape(-1, 1)

        # combine all features
        combined_chunk = np.hstack((
            turbine_type_repeated,
            hub_height_repeated,
            age_repeated,
            wind_speeds
        ))

        # add the data
        combined_features_raw.append(combined_chunk)
        output_raw.append(production_values)

    # combine all data chunks to one array
    combined_features = np.vstack(combined_features_raw)
    output = np.vstack(output_raw)

    # Interpolate missing values (linear interpolation) in pandas
    wind_speed_series = pd.Series(combined_features[:, -1])
    wind_speed_series.interpolate(method='linear', inplace=True)
    combined_features[:, -1] = wind_speed_series.to_numpy()

    # round all values to four decimal places
    combined_features = np.round(combined_features, decimals=4)
    output = np.round(output, decimals=4)
        
    # Normalise numerical features
    scaler_wind = StandardScaler()
    scaler_ages = StandardScaler()
    scaler_hub_heights = StandardScaler()

    # Skalieren der einzelnen Features
    combined_features[:, -1] = scaler_wind.fit_transform(combined_features[:, -1].reshape(-1, 1)).flatten() # scale wind speeds
    combined_features[:, -2] = scaler_ages.fit_transform(combined_features[:, -2].reshape(-1, 1)).flatten()  # scale ages
    combined_features[:, -3] = scaler_hub_heights.fit_transform(combined_features[:, -3].reshape(-1, 1)).flatten()  # scale hub heights
    
    # Convert to PyTorch Dataset
    dataset = WindPowerDataset(combined_features, output)
    
    params = {"batch_size": 128,
              "lr": 0.00010155,
              "number_epochs": 10}
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Train-test split
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    torch.manual_seed(0)
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
    
    # shuffling doesn't matter here, has already taken place during random_split
    train_loader = DataLoader(train_dataset, batch_size=params["batch_size"], shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=params["batch_size"], shuffle=False)
    
    # Model setup
    input_size = combined_features.shape[1]

    # use static instead of dynamic computational graphs
    model = torch.jit.script(MLP(input_size=input_size)).to(device)
    model.to(device)
    
    # Trainings-Konfiguration
    mae_criterion = L1Loss()
    mse_criterion = MSELoss()
    huber_criterion = HuberLoss()
    optimizer = optim.Adam(model.parameters(), lr=params["lr"])

    # Training
    print(f"    Training")
    for epoch in range(params["number_epochs"]):
        print(f"        Epoch {epoch + 1}/{params['number_epochs']}")
        model.train()
        train_loss_mae, train_loss_mse, train_loss_huber = 0, 0, 0

        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            outputs = model(batch_x)
            
            # Calculate metrics for each criterion
            loss_mae = mae_criterion(outputs, batch_y)
            loss_mse = mse_criterion(outputs, batch_y)
            loss_huber = huber_criterion(outputs, batch_y)

            optimizer.zero_grad()
            loss_huber.backward()
            optimizer.step()

            # Accumulate metrics for logging
            train_loss_mae += loss_mae.item()
            train_loss_mse += loss_mse.item()
            train_loss_huber += loss_huber.item()

        train_loss_mae /= len(train_loader)
        train_loss_mse /= len(train_loader)
        train_loss_huber /= len(train_loader)

    # Testen
    print(f"    Testing")
    model.eval()

    test_loss_mae, test_loss_mse, test_loss_huber = 0, 0, 0

    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            preds = model(batch_x)
            
            test_loss_mae += mae_criterion(preds, batch_y).item()
            test_loss_mse += mse_criterion(preds, batch_y).item()
            test_loss_huber += huber_criterion(preds, batch_y).item()

    test_loss_mae /= len(test_loader)
    test_loss_mse /= len(test_loader)
    test_loss_huber /= len(test_loader)
    
    models[lead_time] = model.state_dict()
    
    scalers[lead_time] = {
        "winds": scaler_wind,
        "ages": scaler_ages,
        "hub_heights": scaler_hub_heights
    }

    encoders[lead_time] = encoder

    input_sizes[lead_time] = input_size

    metrics[lead_time] = {
        "Training": {
            "Huber": train_loss_huber,
            "MAE": train_loss_mae,
            "MSE":train_loss_mse,
            "RMSE": np.sqrt(train_loss_mse)
        },
        "Testing": {
            "Huber": test_loss_huber,
            "MAE": test_loss_mae,
            "MSE": test_loss_mse,
            "RMSE": np.sqrt(test_loss_mse)
        },
    }

current_utc = datetime.datetime.now(datetime.timezone.utc).strftime("%Y%m%d_%H%M%S")
folder = f"modelC/parameters_crs_{current_utc}"
os.makedirs(folder, exist_ok=True)

# Save all parameters
torch.save(models, os.path.join(folder, "models.pth"))
joblib.dump(scalers, os.path.join(folder, "scalers.pkl"))
joblib.dump(encoders, os.path.join(folder, "encoders.pkl"))
joblib.dump(input_sizes, os.path.join(folder, "input_sizes.pkl"))
joblib.dump(metrics, os.path.join(folder, "metrics.pkl"))
print("All parameters saved successfully.")

<h1>Now change the used model in the source code of the web app